<a href="https://colab.research.google.com/github/prasad-kamlaskar/incubyte-tech/blob/main/IncubyteTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
source_file_path = r'/home/data/src_customers_01.txt'

In [89]:
import pandas as pd
import sqlite3

In [90]:
#Extraction from the source
src = pd.read_csv(source_file_path, header=0, sep='|').iloc[:,1:]
src.head()

,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,Alex,123457,20101012,20121013,MVD,Paul,SA,USA,6031987,A
1,D,John,123458,20101012,20121013,MVD,Paul,TN,IND,6031987,A
2,D,Mathew,123459,20101012,20121013,MVD,Paul,WAS,PHIL,6031987,A
3,D,Matt,12345,20101012,20121013,MVD,Paul,BOS,NYC,6031987,A
4,D,Jacob,1256,20101012,20121013,MVD,Paul,VIC,AU,6031987,A


In [91]:
#staging data
createTableSTG = '''
CREATE TABLE IF NOT EXISTS CUSTOMER_SR (
  'Customer Name' VARCHAR(255) NOT NULL PRIMARY KEY, 
  'Customer ID' VARCHAR(18) NOT NULL,
  'Customer Open Date' DATE NOT NULL,
  'Last Consulted Date' DATE,
  'Vaccination Type' CHAR(5),
  'Doctor Consulted' CHAR(255),
  'State' CHAR(5),
  'Country' CHAR(5),
  'Post Code' DECIMAL(5,0),
  'Date of Birth' DATE,
  'Active Customer' CHAR(1)
) '''

with sqlite3.connect('/home/data/DM_STG.sqlite3') as conn:
  cur = conn.cursor()
  cur.execute(createTableSTG)
  src.to_sql('CUSTOMER_SR',if_exists='replace', method='multi' , index=False, con=conn)
  conn.commit()
  #Country List for which target tables has to be generated
  cur.execute('SELECT DISTINCT Country FROM CUSTOMER_SR')
  countryList = [x[0] for x in cur.fetchall()]
  print('CountryList : ',countryList)

CountryList :  ['USA', 'IND', 'PHIL', 'NYC', 'AU']


In [92]:
#creating target table
with sqlite3.connect('/home/data/DM_STG.sqlite3') as conn:
  cur = conn.cursor()
  #List of tables - Already exist
  cur = conn.execute("SELECT name FROM sqlite_master where type = 'table' and name like 'Table_%' ")
  tableList = [x[0] for x in cur.fetchall()]
  #print('Existng Tables : ',tableList)
  
  for country in countryList:
    if 'Table_' + country not in tableList:
      createTargetSQL = f"CREATE TABLE Table_{country} AS SELECT * FROM CUSTOMER_SR WHERE Country ='{country}'"
      cur = conn.execute(createTargetSQL)
    else:
      insertTargetSQL = f"INSERT INTO Table_{country} SELECT * FROM CUSTOMER_SR WHERE Country ='{country}'"
      cur = conn.execute(insertTargetSQL)
  conn.commit()


In [105]:
#Final Output
with sqlite3.connect('/home/data/DM_STG.sqlite3') as conn:
  cur = conn.cursor()
  #Table Content
  cur = conn.execute("SELECT name FROM sqlite_master where type = 'table' and name like 'Table_%' ")
  tableList = [x[0] for x in cur.fetchall()]

  for table in tableList:
    print('\n\nTable : ',table)
    target = pd.read_sql('SELECT * FROM '+ table, con=conn)
    display(target)
  



Table :  Table_USA


,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,Alex,123457,20101012,20121013,MVD,Paul,SA,USA,6031987,A




Table :  Table_IND


,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,John,123458,20101012,20121013,MVD,Paul,TN,IND,6031987,A




Table :  Table_PHIL


,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,Mathew,123459,20101012,20121013,MVD,Paul,WAS,PHIL,6031987,A




Table :  Table_NYC


,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,Matt,12345,20101012,20121013,MVD,Paul,BOS,NYC,6031987,A




Table :  Table_AU


,H,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,D,Jacob,1256,20101012,20121013,MVD,Paul,VIC,AU,6031987,A
